<H1> Necessary Libraries to create Dashboard <H1>

In [3]:
!pip install dash
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.0
    Uninstalling Werkzeug-3.0.0:
      Successfully uninstalled Werkzeug-3.0.0


In [4]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

<H3> Downloading the CSV file of SpaceX Launch dataset  <H3>

In [5]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(URL)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [12]:
spacex_df.dtypes

Unnamed: 0                    int64
Flight Number                 int64
Launch Site                  object
class                         int64
Payload Mass (kg)           float64
Booster Version              object
Booster Version Category     object
dtype: object

In [21]:
unique_sites=spacex_df['Launch Site'].unique()

In [20]:
filtered_df = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
filtered_df

,Launch Site,class
0,CCAFS LC-40,0.269231
1,CCAFS SLC-40,0.428571
2,KSC LC-39A,0.769231
3,VAFB SLC-4E,0.400000


In [19]:
filtered_df = spacex_df[spacex_df['Launch Site']=='CCAFS LC-40']
fig = px.pie(filtered_df,
        names='class',
        title='Total Success Launches by Site')
fig.show()

In [86]:
spacex_df['Payload Mass (kg)'].max()

9600.0

<H2> Creating the DASH App <H2>

In [98]:
#Create app
app = dash.Dash(__name__)
#Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True
#Layout Section of Dash
#Task 1 Add the Title to the Dashboard
#Task 1 Add the Title to the Dashboard
app.layout = html.Div(children=[html.H1('SpaceX launch Records Dashboard',
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
                                #Dropdown to select year
                    html.Div([
                        dcc.Dropdown(id='site-dropdown',options=[{"label":"All Sites","value": "All"},]+[{"label":site,"value": site} for site in spacex_df['Launch Site'].unique()],
                                     placeholder='Select a launch Site here', value = 'None',style={'width': '80%', 'color': 'blue','font-size': 20,'text-align-last': 'center', 'padding': '3px'}, searchable=True)
                    ]),
                    #output figure 1
                    html.Div([dcc.Graph(id='success-pie-chart', className='chart-grid', style={'display': 'flex'}),
                    ]),

                    html.Div([
                        dcc.RangeSlider(id='payload-slider',min=spacex_df['Payload Mass (kg)'].min(), max=spacex_df['Payload Mass (kg)'].max(), step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 9600: '9600'},value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()])
                    ]),

                    #output figure 2
                    html.Div([dcc.Graph(id='success-payload-scatter-chart', className='chart-grid', style={'display': 'flex'}),
                    ])
                               ]
                     )

In [99]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'All':
        filtered_df = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
        fig = px.pie(filtered_df, values='class',
        names='Launch Site',
        title='Total Success Launches by Site')
        return fig
    elif entered_site in spacex_df['Launch Site'].unique():
      filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
      fig = px.pie(filtered_df, names='class', title=f'Total Success Launches for site {entered_site}')
      return fig
    else:
      return True
        # return the outcomes piechart for a selected site

In [100]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')])

def get_scatter_plot(entered_site, payload_mass):
    min_payload, max_payload = payload_mass
    if entered_site == 'All':
        payload_df=spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]
        fig = px.scatter(payload_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',title='Correlation between Payload and Success for all Sites')
        return fig
    elif entered_site in spacex_df['Launch Site'].unique():
      site_df = spacex_df[spacex_df['Launch Site']==entered_site]
      payload_df=site_df[(site_df['Payload Mass (kg)'] >= min_payload) & (site_df['Payload Mass (kg)'] <= max_payload)]
      fig = px.scatter(payload_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',title=f'Correlation between Payload and Success for site {entered_site}')
      return fig
    else:
      return True
        # return the outcomes piechart for a selected site

In [101]:
if __name__ == '__main__':
    app.run(jupyter_mode="external", port=8050, debug=False)

Dash app running on:


<IPython.core.display.Javascript object>